<a href="https://colab.research.google.com/github/debar2001/ISM-Project-XSS-Attack-Detection-using-Neural-Network-Model/blob/main/ISM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import keras
import cv2

### **First mount your Drive using the folder tab shown in the left side of the screen than create a Data Frame where you will read the dataset.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset_ISM/XSS_dataset.csv')

In [ ]:
df.head()

,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


### **Only get sentence and labels column**

In [ ]:
df=df[df.columns[-2:]]
df.head()

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


### Get Sentences data from data **frame**

In [ ]:
sentences=df['Sentence'].values
sentences[1]

'<tt onmouseover="alert(1)">test</tt>'

In [ ]:
print(len(sentences))

13686


### **Convert the sentences to ASCII**

In [ ]:
def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        
        
        """Some characters have values very big e.d 8221 and some are chinese letters
        I am removing letters having values greater than 8222 and for rest greater 
        than 128 and smaller than 8222 assigning them values so they can easily be normalized"""
       
        if(ord(i)<8222):      # ” has ASCII of 8221
            
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)
            
            
            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)
                
            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)
                
                
            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)
                
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)
            
            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)
                
                
            """
            If values less than 128 store them else discard them
            """
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))
    
            else:
                    pass
            

    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)


    return zer

### **Send each sentence to be converted to ASCII**

In [ ]:
arr=np.zeros((len(sentences),100,100))

for i in range(len(sentences)):
    
    image=convert_to_ascii(sentences[i])

    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128

        
    arr[i]=image

In [ ]:
print("Input data shape : ", arr.shape)

Input data shape :  (13686, 100, 100)


### **Reshape data for input to CNN**

In [ ]:
data = arr.reshape(arr.shape[0], 100, 100, 1)
data.shape

(13686, 100, 100, 1)

In [ ]:
y=df['Label'].values

### **Split into train/test data**

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)

### **Import libraries for Making Model**

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization

### **A basic CNN Model,** 
### **Number of layers = 11,** 
### **Number of Convolutional layer = 3**

In [ ]:
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 64)        640       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 128)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 10, 10, 256)     

### **Model training**

In [ ]:
batch_size = 128
num_epoch = 10
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY)
                     )

Epoch 1/10
86/86 [==============================] - 15s 170ms/step - loss: 0.4476 - accuracy: 0.7872 - val_loss: 0.3744 - val_accuracy: 0.8276
Epoch 2/10
86/86 [==============================] - 14s 165ms/step - loss: 0.2064 - accuracy: 0.9193 - val_loss: 0.0769 - val_accuracy: 0.9766
Epoch 3/10
86/86 [==============================] - 14s 165ms/step - loss: 0.0648 - accuracy: 0.9812 - val_loss: 0.0603 - val_accuracy: 0.9825
Epoch 4/10
86/86 [==============================] - 14s 165ms/step - loss: 0.0575 - accuracy: 0.9837 - val_loss: 0.0517 - val_accuracy: 0.9858
Epoch 5/10
86/86 [==============================] - 14s 165ms/step - loss: 0.0466 - accuracy: 0.9867 - val_loss: 0.0552 - val_accuracy: 0.9854
Epoch 6/10
86/86 [==============================] - 14s 165ms/step - loss: 0.0438 - accuracy: 0.9883 - val_loss: 0.0499 - val_accuracy: 0.9865
Epoch 7/10
86/86 [==============================] - 14s 166ms/step - loss: 0.0404 - accuracy: 0.9889 - val_loss: 0.0472 - val_accuracy: 0.9883

### **Predict for test set**

In [ ]:
pred=model.predict(testX)

### **Threshold values predicted**

In [ ]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

### **Count True predicted and wrong predicted**

In [ ]:
true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1
        
print("True predicted :: ", true)
print("False prediction :: ", false)

True predicted ::  2709
False prediction ::  29


### **Number of attack and benign data in test set**

In [ ]:
attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
    else:
        benign+=1

print("Attack data in test set :: ", attack)
print("Benign data in test set :: ", benign)

Attack data in test set ::  1478
Benign data in test set ::  1260


### **Takes True positive, true negative, false positive and false negative as input**
### **Returns Accuracy**

In [ ]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

### **Takes True Positive and false positive**
### **Returns Precision**

In [ ]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

###**Takes True Positive and false negative**
###**Returns Recall**

In [ ]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

###**Takes true values and predicted**
###**Returns Accuracy, precision and recall**

In [ ]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [ ]:
accuracy,precision,recall=confusion_matrix(testY,pred)

### **Get the value for different Performance Metrics**

In [ ]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 Accuracy : 0.9894083272461651 
 Precision : 0.9878787878787879 
 Recall : 0.9925575101488497
